# Handling image logs with dlisio

This notebook builds on the plotting work from Brendon Hall.

For using mayavi - make sure you install dependencies and on Windows 10 you will need the visual basic built tools from https://visualstudio.microsoft.com/visual-cpp-build-tools/

In [ ]:
%matplotlib inline

import re
import os
import pandas as pd
import dlisio
import matplotlib.pyplot as plt
import numpy as np
import numpy.lib.recfunctions as rfn
from mayavi import mlab

Load the [NGI image](https://drive.google.com/file/d/1oY-a7d-lIG8mBNE52BGOeCmq6u1kbFkx/view?usp=sharing) from the Pharos well in Poseidon.  Source: https://nopims.dmp.wa.gov.au/Nopims/Search/WellDetails#

In [ ]:
filepath = r"ConocoPhillips Pharos-1 NGI Processed Images Static_Dynamic.dlis"

## Which curves are available?

### We can quickly pull the metadata of the file and the list of curves per frame to find the image log we are looking for.

In [ ]:
with dlisio.dlis.load(filepath) as file:
    for d in file:
        for origin in d.origins:
            print(origin.describe())
        for fram in d.frames:
            print(fram.describe())

We will pull the depth track and the image track.  From the depth track we need the last array value so we can plot the limit.

In [ ]:
for d in file:
    for frame in d.frames:
        for channel in frame.channels:
            print(channel.describe())

In [ ]:
with dlisio.dlis.load(filepath) as file:
    for d in file:
        depth_channels = d.find('CHANNEL', 'TDEP')
        for channel in depth_channels:
            print(channel.name)
            depth_array = channel.curves()
            max_depth = depth_array[-1]
        image_channels = d.find('CHANNEL','NGI DYNAMIC.')
        for channel in image_channels:
            print(channel.name)
            image_array = channel.curves()

In [ ]:
max_depth

In [ ]:
df = pd.DataFrame(image_array, index=depth_array)

In [ ]:
df

In [ ]:
plt.figure(figsize = (24,30))
image_array[image_array == -9999.] = np.nan
limit = int(100000 + max_depth)
plt.imshow(image_array[100000:limit,:], cmap='YlOrBr')
plot = plt.colorbar()

### Plot NGI wellbore image in 3D 

In [ ]:
from PIL import Image

myarr = image_array[100000:100800,:]
max_value = np.nanmax(myarr)
myarr = myarr / max_value
im = Image.fromarray(np.uint8(plt.cm.YlOrBr(myarr)*255))


im.save('test_image.png')

In [ ]:
%gui qt

cyl = mlab.pipeline.builtin_surface()
cyl_surf = mlab.pipeline.surface(cyl)

cyl.source = 'cylinder'
cyl.data_source.resolution = 64
cyl.data_source.height = 8.0

img = mlab.pipeline.open('./test_image.png')
cyl_surf.actor.enable_texture = True
cyl_surf.actor.tcoord_generator_mode = 'cylinder'
cyl_surf.actor.texture_source_object = img
cyl_surf.actor.tcoord_generator.prevent_seam = False

Plot the wellbore image wrapped around a cylinder in 3D.  After running the cell below you may get a pop up window reporting an error.  Close this window and the image log in 3D will appear.

In [13]:
from mayavi import mlab
from tvtk.api import tvtk # python wrappers for the C++ vtk ecosystem

def auto_sphere(image_file):
    # create a figure window (and scene)
    fig = mlab.figure(size=(600, 600))

    # load and map the texture
    img = tvtk.JPEGReader()
    img.file_name = image_file
    texture = tvtk.Texture(input_connection=img.output_port, interpolate=1)
    # (interpolate for a less raster appearance when zoomed in)

    # use a TexturedSphereSource, a.k.a. getting our hands dirty
    R = 1
    Nrad = 180

    # create the sphere source with a given radius and angular resolution
    sphere = tvtk.TexturedSphereSource(radius=R, theta_resolution=Nrad,
                                       phi_resolution=Nrad)

    # assemble rest of the pipeline, assign texture    
    sphere_mapper = tvtk.PolyDataMapper(input_connection=sphere.output_port)
    sphere_actor = tvtk.Actor(mapper=sphere_mapper, texture=texture)
    fig.scene.add_actor(sphere_actor)

image_file = './blue_marble_spherical.jpg'
auto_sphere(image_file)
mlab.show()